## Pubmed search validation statistics
GNU General Public License v3.0 - Robert Ietswaart

### Citation:
Robert Ietswaart<sup>\*,#</sup>, Seda Arat<sup>\*,#</sup>, Amanda X. Chen<sup>\*</sup>, 
Saman Farahmand<sup>\*</sup>, Bumjun Kim, William DuMouchel, 
Duncan Armstrong, Alexander Fekete, Jeffrey J. Sutherland<sup>#</sup>, Laszlo Urban<sup>#</sup>  
*Machine learning guided association of adverse drug reactions with in vitro target-based 
pharmacology* (2019), [BioRxiv; 750950](https://www.biorxiv.org/content/10.1101/750950v2).

In [1]:
import re
import copy
import numpy as np
import pickle as pkl
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42
import seaborn as sns
from scipy.stats import fisher_exact, chi2_contingency,  mannwhitneyu, ks_2samp
from statsmodels.stats.multitest import fdrcorrection

In [2]:
path='/Users/horizon/Documents/HMS/Novartis2018Hackathon/PCS/Saman/'
filename='Full_result_ADRmesh_Genemesh_pubmed_NEW_106.csv'

ATpmid = pd.read_csv(path+filename,sep='\t')#
ATpmid.head()

,HLGT,gene,mesh_tr,HLGT_N,mesh_N,intersect_N,flag
0,abdominal hernias and other abdominal wall con...,ABCB11,"ATP Binding Cassette Transporter, Subfamily B,...",32021,554,0,False
1,abdominal hernias and other abdominal wall con...,ACHE,Acetylcholinesterase,32021,21600,1,False
2,abdominal hernias and other abdominal wall con...,ADORA1,"Receptor, Adenosine A1",32021,1347,0,False
3,abdominal hernias and other abdominal wall con...,ADORA2A,"Receptor, Adenosine A2A",32021,2325,0,False
4,abdominal hernias and other abdominal wall con...,ADORA3,"Receptor, Adenosine A3",32021,746,0,False


In [3]:
mapper={'HLGT_N':'N_A','mesh_N':'N_T','intersect_N':'N_AT','flag':'RF_pred'}
ATpmid=ATpmid.rename(columns=mapper)
len(ATpmid)

26925

In [4]:
dtemp=ATpmid[ATpmid['RF_pred']==False]
dtemp=dtemp.drop_duplicates(['HLGT','gene'])
print(len(dtemp[dtemp['N_AT']>0]),'/',len(dtemp),
      len(dtemp[dtemp['N_AT']>0])/len(dtemp) ,'of negatives have literature co-occurrence')

14044 / 26705 0.5258940273357049 of negatives have literature co-occurrence


In [5]:
dtemp=ATpmid[ATpmid['RF_pred']==True]
dtemp=dtemp.drop_duplicates(['HLGT','gene'])
print(len(dtemp[dtemp['N_AT']>0]),'/',len(dtemp),
      len(dtemp[dtemp['N_AT']>0])/len(dtemp) ,'of predictions have literature co-occurrence')

145 / 219 0.6621004566210046 of predictions have literature co-occurrence


## Calculate lift

In [6]:
N_all_pmids=29138919
#information retrieved from:
#https://www.nlm.nih.gov/bsd/licensee/baselinestats.html
#https://www.nlm.nih.gov/bsd/licensee/2019_stats/2019_LO.html


In [7]:
ATpmid['lift']=N_all_pmids*ATpmid['N_AT']/(ATpmid['N_A']*ATpmid['N_T'])

In [8]:
dtemp=ATpmid[ATpmid['lift']>1]
print(len(dtemp[dtemp['RF_pred']==False])/len(ATpmid[ATpmid['RF_pred']==False]),
      'of negatives have positive lift')
print(len(dtemp[dtemp['RF_pred']==True])/len(ATpmid[ATpmid['RF_pred']==True]),
      'of predictions have positive lift')
###NB results for lift and FE odds ratio is exactly the same

0.12967609061973412 of negatives have positive lift
0.22272727272727272 of predictions have positive lift


In [9]:
print(np.median(ATpmid[ATpmid['RF_pred']==True]['lift']))
print(np.median(ATpmid[ATpmid['RF_pred']==False]['lift']))
print(np.mean(ATpmid[ATpmid['RF_pred']==True]['lift']))
print(np.mean(ATpmid[ATpmid['RF_pred']==False]['lift']))
# print(max(ATpmid[ATpmid['RF_pred']==True]['lift']))
# print(max(ATpmid[ATpmid['RF_pred']==False]['lift']))

0.16186094081962868
0.043996415841097675
1.2636508842450893
0.6232813870384358


In [10]:
print('fold',np.median(ATpmid[ATpmid['RF_pred']==True]['lift'])/np.median(ATpmid[ATpmid['RF_pred']==False]['lift']))

fold 3.6789574269918615


In [11]:
#mannwhitneyu test = Mann Whitney U test = Wilcoxon rank sum test (not paired)
#https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.mannwhitneyu.html#scipy.stats.mannwhitneyu
#with corrects for ties (=equal rankings) and continuity correction since our data is discrete

In [12]:
U , pval = mannwhitneyu(ATpmid[ATpmid['RF_pred']==True]['lift'],
                        ATpmid[ATpmid['RF_pred']==False]['lift'],
                        use_continuity=True,
                        alternative='two-sided')

U, pval

(3403522.5, 1.7716158369081986e-05)

In [13]:
def get_contingency_table(universe,Total_condition1,Total_condition2,Overlap_cond12):
    ct = [[1, 2], [3, 4]]#contingency table
    ct[0][0] = Overlap_cond12
    ct[0][1] = Total_condition2-Overlap_cond12
    ct[1][0] = Total_condition1-Overlap_cond12
    ct[1][1] = universe-Total_condition1-Total_condition2+Overlap_cond12
    return ct


## FE / chi2 test to see if Pubmed retrieval rate is higher than over background

In [14]:
N_all_ATs = len(ATpmid)
dtemp=ATpmid[ATpmid['RF_pred']==True]
N_RF_pred = len(dtemp)
dtemp = ATpmid[ATpmid['N_AT']>0]
N_pmpos = len(dtemp)
N_RF_pred_pmpos=len(dtemp[dtemp['RF_pred']==True])

print(N_all_ATs,
      N_RF_pred,
      N_pmpos,
      N_RF_pred_pmpos)
cont_table = get_contingency_table(N_all_ATs,
                                   N_RF_pred,
                                   N_pmpos,
                                   N_RF_pred_pmpos)    

OR , pval = fisher_exact(cont_table)
print('Fisher Exact odds ratio',OR,'pvalue',pval)
c2, pval, dof, ex = chi2_contingency(cont_table, correction=True)
print('Chi2 statistic',c2,'pvalue',pval,'degrees of freedom',dof)
print('contingency table \n', cont_table)
print('expected contingency table under independence null hypothesis \n', ex)


26925 220 14189 145
Fisher Exact odds ratio 1.7429459793031425 pvalue 7.813160795292085e-05
Chi2 statistic 15.00035262475823 pvalue 0.00010749108914179624 degrees of freedom 1
contingency table 
 [[145, 14044], [75, 12661]]
expected contingency table under independence null hypothesis 
 [[  115.93611885 14073.06388115]
 [  104.06388115 12631.93611885]]


### Idem with dropping duplicates

In [15]:
N_all_ATs = len(ATpmid.drop_duplicates(['HLGT','gene']))
dtemp=ATpmid[ATpmid['RF_pred']==True]
dtemp=dtemp.drop_duplicates(['HLGT','gene'])#some assays (eg hERG binding, hERG QP) are mapped to same gene:
#this will duplicate the pubmed retrieval query
N_RF_pred = len(dtemp)
dtemp = ATpmid[ATpmid['N_AT']>0]
dtemp=dtemp.drop_duplicates(['HLGT','gene'])#some assays (eg hERG binding, hERG QP) are mapped to same gene:

#this will duplicate the pubmed retrieval query
N_pmpos = len(dtemp)
N_RF_pred_pmpos=len(dtemp[dtemp['RF_pred']==True])
                    
print(N_all_ATs,
      N_RF_pred,
      N_pmpos,
      N_RF_pred_pmpos)
cont_table = get_contingency_table(N_all_ATs,
                                   N_RF_pred,
                                   N_pmpos,
                                   N_RF_pred_pmpos)    

OR , pval = fisher_exact(cont_table)
print('Fisher Exact odds ratio',OR,'pvalue',pval)
c2, pval, dof, ex = chi2_contingency(cont_table, correction=True)
print('Chi2 statistic',c2,'pvalue',pval,'degrees of freedom',dof)
print('contingency table \n', cont_table)
print('expected contingency table under independence null hypothesis \n', ex)

26924 219 14189 145
Fisher Exact odds ratio 1.7664993033477796 pvalue 5.6019239437360516e-05
Chi2 statistic 15.624874641076845 pvalue 7.723191520112344e-05 degrees of freedom 1
contingency table 
 [[145, 14044], [74, 12661]]
expected contingency table under independence null hypothesis 
 [[  115.41342297 14073.58657703]
 [  103.58657703 12631.41342297]]


In [16]:
path = '/Users/horizon/Documents/HMS/Novartis2018Hackathon/PCS/'
filename = 'Pubmed_search_validation_statistics_rev.csv'
ATpmid.to_csv(path+filename)